In [ ]:
import requests
import PIL.Image
import io

访问 https://ssl.ptlogin2.qq.com/ptqrshow?appid=716027609&e=2&l=M&s=3&d=72&v=4&t=0.1957881457063695&daid=383&pt_3rd_aid=102013353&u1=https%3A%2F%2Fgraph.qq.com%2Foauth2.0%2Flogin_jump

In [ ]:
resp = requests.get(
    url="https://ssl.ptlogin2.qq.com/ptqrshow",
    params={
        "appid": 716027609,
        "e": 2,
        "l": "M",
        "s": 3,
        "d": 72,
        "v": 4,
        "t": 0.37961976952352483,
        "daid": 383,
        "pt_3rd_aid": 102013353,
        "u1": "https://graph.qq.com/oauth2.0/login_jump",
    },
)


display(PIL.Image.open(io.BytesIO(resp.content)))
print(resp.headers)

cookies = resp.cookies.get_dict()
print(cookies)

获取pt_local_token，访问 https://xui.ptlogin2.qq.com/cgi-bin/xlogin?target=self&appid=522005705&daid=4&s_url=https%3A%2F%2Fwx.mail.qq.com%2Flist%2Freadtemplate%3Fname%3Dlogin_jump.html%26target%3D&style=25&low_login=1&proxy_url=https://mail.qq.com/proxy.html&need_qr=0&hide_border=1&border_radius=0&self_regurl=https%3A%2F%2Freg.mail.qq.com&app_id=11005?t=regist&pt_feedback_link=http://support.qq.com/discuss/350_1.shtml&css=https://res.mail.qq.com/zh_CN/htmledition/style/ptlogin_input_for_xmail.css&enable_qlogin=0

In [ ]:
resp = requests.get(
    url="https://xui.ptlogin2.qq.com/cgi-bin/xlogin",
    params={
        "appid": 716027609,
        "daid": 383,
        "style": 33,
        "login_text": "登录",
        "hide_title_bar": 1,
        "hide_border": 1,
        "target": "self",
        "s_url": "https://graph.qq.com/oauth2.0/login_jump",
        "pt_3rd_aid": 102013353,
        "pt_feedback_link": "https://support.qq.com/products/77942?customInfo=.appid102013353",
        "theme": 10,
        "verify_theme": "",
    },
    cookies=cookies,
)

print(resp.text)
print(resp.headers)

cookies = cookies | resp.cookies.get_dict()
print(cookies)

pt_local_token = resp.cookies.get("pt_local_token")
print(pt_local_token)

访问 https://localhost.ptlogin2.qq.com:4301/pt_get_uins?callback=ptui_getuins_CB&r=0.7544340024793896&pt_local_tk=pt_local_tk

In [ ]:
def random_float(length):

    import random
    
    # 返回小数点后有length位并以0开头的随机数
    n = random.random() / 10 ** (length + 1)

    print(n)

    return n

resp = requests.get(
    url="https://localhost.ptlogin2.qq.com:4301/pt_get_uins",
    params={
        "callback": "ptui_getuins_CB",
        "r": random_float(16),
        "pt_local_tk": pt_local_token,
    },
    headers={
        "Referer": "https://xui.ptlogin2.qq.com/",
    },
    cookies=cookies,
)

print(resp.text)

import re

# 从text取出[{...}]
m = re.search(r"\[.*\]", resp.text)

print(m.group())

import json

# 将字符串转换为json对象

m = json.loads(m.group())

print(m)

cookies = cookies | resp.cookies.get_dict()

利用 QQuin 与 pt_local_token 获取 QQClientkey。
https://localhost.ptlogin2.qq.com:4301/pt_get_st?clientuin=QQUin&r=0.8134579633763475&pt_local_tk=pt_local_token&callback=__jp0

请求标头带上 Referer

Referer: https://xui.ptlogin2.qq.com/

In [ ]:
resp = requests.get(
    url="https://localhost.ptlogin2.qq.com:4301/pt_get_st",
    params={
        "clientuin": m[0]["uin"],
        "r": random_float(16),
        "pt_local_tk": pt_local_token,
        "callback": "__jp0",
    },
    cookies=cookies,
    headers={"Referer": "https://xui.ptlogin2.qq.com/"},
)

print(resp.text)

cookies = cookies | resp.cookies.get_dict()

print(cookies)

获取 ptsigx

Request URL:

https://ssl.ptlogin2.qq.com/jump?clientuin=QQUin&keyindex=9&pt_aid=716027609&daid=383&u1=https%3A%2F%2Fgraph.qq.com%2Foauth2.0%2Flogin_jump&pt_local_tk=633103212&pt_3rd_aid=102013353&ptopt=1&style=40

请求标头带上 Referer 与 Cookie（QQClientkey）

Referer: https://xui.ptlogin2.qq.com/
Cookie: QQClientkey

In [ ]:
resp = requests.get(
    url="https://ssl.ptlogin2.qq.com/jump",
    params={
        "clientuin": m[0]["uin"],
        "keyindex": 19,
        "pt_aid": 716027609,
        "daid": 383,
        "u1": "https://graph.qq.com/oauth2.0/login_jump",
        "pt_local_tk": pt_local_token,
        "pt_3rd_aid": 102013353,
        "ptopt": 1,
        "style": 40,
    },
    headers={"Referer": "https://xui.ptlogin2.qq.com/"},
    cookies=cookies,
    allow_redirects=False,
)

print(resp.text)
print(resp.headers)

cookies = cookies | resp.cookies.get_dict()

print(json.dumps(cookies, indent=4))
print(resp.cookies.get_dict())

check_sig_url = eval(resp.text.replace('ptui_qlogin_CB', ''))[1]

print(check_sig_url)

skey和pskey


https://ssl.ptlogin2.graph.qq.com/check_sig?pttype=2&uin=1010553892&service=jump&nodirect=0&ptsigx=852bc2705d326a5d68e9e2ae99d607d09be91b08d1a0626d97198ede920b979b71f9b0e551b6d9b4d2cc906ff662fc510169210d7a46a588a9db503050841003ba3ebf2834bb4fd2&s_url=https%3A%2F%2Fgraph.qq.com%2Foauth2.0%2Flogin_jump&f_url=&ptlang=2052&ptredirect=100&aid=716027609&daid=383&j_later=0&low_login_hour=0&regmaster=0&pt_login_type=-1&pt_aid=716027609&pt_aaid=0&pt_light=0&pt_3rd_aid=102013353


Referer: https://xui.ptlogin2.qq.com/

In [ ]:


resp = requests.get(
    url=check_sig_url,
    headers={"Referer": "https://xui.ptlogin2.qq.com/"},
    # cookies=cookies,
    allow_redirects=False,
)
print(resp.status_code)
print(resp.text)

print(resp.headers)
print(resp.cookies.get_dict())

cookies = cookies | resp.cookies.get_dict()

print(json.dumps(cookies, indent=4))

